In [1]:
from keras.models import Model, Input, save_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras import losses
import h5py

DATA_FILE = 'data.h5'

Using TensorFlow backend.


In [2]:
with h5py.File(DATA_FILE, 'r') as f:
    
    images_training = list(f['images_training'])
    labels_training = list(f['label_training'])
        
    images_testing = list(f['images_testing'])
    labels_testing = list(f['label_testing'])

In [36]:
activation = 'relu'

input_shape = (128, 128, 3)
pic_input = Input(shape=input_shape)

img_stack = Conv2D(64, (3, 3), name='convolution0', padding='same', activation=activation)(pic_input)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = Conv2D(64, (3, 3), name='convolution1', padding='same', activation=activation)(img_stack)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = Conv2D(32, (3, 3), name='convolution2', padding='same', activation=activation)(img_stack)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = Conv2D(32, (3, 3), name='convolution3', padding='same', activation=activation)(img_stack)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = Flatten()(img_stack)
img_stack = Dropout(0.25)(img_stack)

dense_stack = Dense(64, activation=activation, name='dense0')(img_stack)
dense_stack = Dropout(0.25)(dense_stack)
dense_stack = Dense(32, activation=activation, name='dense1')(dense_stack)
dense_stack = Dense(1, name='output', activation='tanh')(dense_stack)

adam = Adam()
model = Model(inputs=[pic_input], outputs=dense_stack)
model.compile(optimizer=adam, loss='mse')


In [37]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 128, 128, 3)       0         
_________________________________________________________________
convolution0 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 64, 64, 64)        0         
_________________________________________________________________
convolution1 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
max_pooling2d_42 (MaxPooling (None, 32, 32, 64)        0         
_________________________________________________________________
convolution2 (Conv2D)        (None, 32, 32, 32)        18464     
_________________________________________________________________
max_pooling2d_43 (MaxPooling (None, 16, 16, 32)        0         
__________

In [38]:
from generator import DriveDataGenerator

plateau_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.0001, verbose=1)
model_checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)
tensorboard_callback = TensorBoard(log_dir='./logs')

resize_dims = (input_shape[0], input_shape[1])
train_generator = DriveDataGenerator(images_training, labels_training, resize_dims=resize_dims)
validation_generator = DriveDataGenerator(images_testing, labels_testing, resize_dims=resize_dims)

model.fit_generator(generator=train_generator, validation_data=validation_generator,
                    epochs=500, verbose=1, 
                    callbacks=[model_checkpoint, tensorboard_callback])

Epoch 1/500
117/117 [==============================] - 9s 80ms/step - loss: 0.7997 - val_loss: 0.6373
Epoch 2/500
117/117 [==============================] - 9s 75ms/step - loss: 0.4006 - val_loss: 0.1539
Epoch 3/500
117/117 [==============================] - 9s 75ms/step - loss: 0.1455 - val_loss: 0.1218
Epoch 4/500
117/117 [==============================] - 9s 75ms/step - loss: 0.1268 - val_loss: 0.1060
Epoch 5/500
117/117 [==============================] - 9s 75ms/step - loss: 0.1165 - val_loss: 0.1368
Epoch 6/500
117/117 [==============================] - 9s 75ms/step - loss: 0.1320 - val_loss: 0.1101
Epoch 7/500
117/117 [==============================] - 9s 76ms/step - loss: 0.1181 - val_loss: 0.1073
Epoch 8/500
117/117 [==============================] - 9s 78ms/step - loss: 0.1119 - val_loss: 0.1024
Epoch 9/500
117/117 [==============================] - 10s 88ms/step - loss: 0.1069 - val_loss: 0.1019
Epoch 10/500
117/117 [==============================] - 10s 84ms/step - loss: 0.1

117/117 [==============================] - 9s 81ms/step - loss: 0.0789 - val_loss: 0.0833
Epoch 81/500
117/117 [==============================] - 9s 80ms/step - loss: 0.0804 - val_loss: 0.0736
Epoch 82/500
117/117 [==============================] - 9s 80ms/step - loss: 0.0766 - val_loss: 0.0769
Epoch 83/500
117/117 [==============================] - 9s 80ms/step - loss: 0.0759 - val_loss: 0.0768
Epoch 84/500
117/117 [==============================] - 9s 79ms/step - loss: 0.0783 - val_loss: 0.0811
Epoch 85/500
117/117 [==============================] - 9s 80ms/step - loss: 0.0830 - val_loss: 0.0725
Epoch 86/500
117/117 [==============================] - 9s 79ms/step - loss: 0.0760 - val_loss: 0.0782
Epoch 87/500
117/117 [==============================] - 9s 79ms/step - loss: 0.0767 - val_loss: 0.0733
Epoch 88/500
117/117 [==============================] - 9s 80ms/step - loss: 0.0746 - val_loss: 0.0730
Epoch 89/500
117/117 [==============================] - 10s 82ms/step - loss: 0.0813 -

117/117 [==============================] - 10s 82ms/step - loss: 0.0678 - val_loss: 0.0763
Epoch 160/500
117/117 [==============================] - 9s 80ms/step - loss: 0.0674 - val_loss: 0.0695
Epoch 161/500
117/117 [==============================] - 9s 80ms/step - loss: 0.0645 - val_loss: 0.0687
Epoch 162/500
117/117 [==============================] - 9s 80ms/step - loss: 0.0661 - val_loss: 0.0638
Epoch 163/500
117/117 [==============================] - 9s 80ms/step - loss: 0.0683 - val_loss: 0.0709
Epoch 164/500
117/117 [==============================] - 9s 81ms/step - loss: 0.0650 - val_loss: 0.0690
Epoch 165/500
117/117 [==============================] - 9s 80ms/step - loss: 0.0662 - val_loss: 0.0632
Epoch 166/500
117/117 [==============================] - 9s 80ms/step - loss: 0.0623 - val_loss: 0.0673
Epoch 167/500
117/117 [==============================] - 9s 80ms/step - loss: 0.0648 - val_loss: 0.0645
Epoch 168/500
117/117 [==============================] - 9s 80ms/step - loss:

117/117 [==============================] - 12s 104ms/step - loss: 0.0564 - val_loss: 0.0695
Epoch 238/500
117/117 [==============================] - 10s 82ms/step - loss: 0.0591 - val_loss: 0.0674
Epoch 239/500
117/117 [==============================] - 10s 87ms/step - loss: 0.0591 - val_loss: 0.0657
Epoch 240/500
117/117 [==============================] - 10s 86ms/step - loss: 0.0560 - val_loss: 0.0614
Epoch 241/500
117/117 [==============================] - 10s 83ms/step - loss: 0.0578 - val_loss: 0.0654
Epoch 242/500
117/117 [==============================] - 10s 85ms/step - loss: 0.0609 - val_loss: 0.0643
Epoch 243/500
117/117 [==============================] - 10s 83ms/step - loss: 0.0586 - val_loss: 0.0666
Epoch 244/500
117/117 [==============================] - 10s 86ms/step - loss: 0.0573 - val_loss: 0.0655
Epoch 245/500
117/117 [==============================] - 10s 86ms/step - loss: 0.0578 - val_loss: 0.0678
Epoch 246/500
117/117 [==============================] - 11s 91ms/st

117/117 [==============================] - 14s 123ms/step - loss: 0.0533 - val_loss: 0.0648
Epoch 315/500
117/117 [==============================] - 10s 86ms/step - loss: 0.0503 - val_loss: 0.0599
Epoch 316/500
117/117 [==============================] - 10s 86ms/step - loss: 0.0550 - val_loss: 0.0672
Epoch 317/500
117/117 [==============================] - 10s 86ms/step - loss: 0.0543 - val_loss: 0.0621
Epoch 318/500
117/117 [==============================] - 11s 91ms/step - loss: 0.0506 - val_loss: 0.0631
Epoch 319/500
117/117 [==============================] - 11s 91ms/step - loss: 0.0515 - val_loss: 0.0629
Epoch 320/500
117/117 [==============================] - 10s 88ms/step - loss: 0.0512 - val_loss: 0.0661
Epoch 321/500
117/117 [==============================] - 11s 90ms/step - loss: 0.0515 - val_loss: 0.0720
Epoch 322/500
117/117 [==============================] - 10s 87ms/step - loss: 0.0515 - val_loss: 0.0658
Epoch 323/500
117/117 [==============================] - 10s 87ms/st

117/117 [==============================] - 10s 83ms/step - loss: 0.0483 - val_loss: 0.0680
Epoch 393/500
117/117 [==============================] - 9s 79ms/step - loss: 0.0474 - val_loss: 0.0637
Epoch 394/500
117/117 [==============================] - 9s 80ms/step - loss: 0.0461 - val_loss: 0.0646
Epoch 395/500
117/117 [==============================] - 10s 82ms/step - loss: 0.0501 - val_loss: 0.0659
Epoch 396/500
117/117 [==============================] - 9s 80ms/step - loss: 0.0505 - val_loss: 0.0640
Epoch 397/500
117/117 [==============================] - 10s 81ms/step - loss: 0.0492 - val_loss: 0.0660
Epoch 398/500
117/117 [==============================] - 10s 82ms/step - loss: 0.0490 - val_loss: 0.0658
Epoch 399/500
117/117 [==============================] - 10s 82ms/step - loss: 0.0457 - val_loss: 0.0718
Epoch 400/500
117/117 [==============================] - 9s 81ms/step - loss: 0.0451 - val_loss: 0.0634
Epoch 401/500
117/117 [==============================] - 9s 80ms/step - l

117/117 [==============================] - 9s 81ms/step - loss: 0.0456 - val_loss: 0.0662
Epoch 471/500
117/117 [==============================] - 9s 80ms/step - loss: 0.0449 - val_loss: 0.0674
Epoch 472/500
117/117 [==============================] - 9s 79ms/step - loss: 0.0395 - val_loss: 0.0657
Epoch 473/500
117/117 [==============================] - 10s 87ms/step - loss: 0.0427 - val_loss: 0.0652
Epoch 474/500
117/117 [==============================] - 9s 80ms/step - loss: 0.0416 - val_loss: 0.0615
Epoch 475/500
117/117 [==============================] - 9s 79ms/step - loss: 0.0434 - val_loss: 0.0611
Epoch 476/500
117/117 [==============================] - 9s 80ms/step - loss: 0.0455 - val_loss: 0.0643
Epoch 477/500
117/117 [==============================] - 9s 80ms/step - loss: 0.0401 - val_loss: 0.0635
Epoch 478/500
117/117 [==============================] - 9s 81ms/step - loss: 0.0443 - val_loss: 0.0652
Epoch 479/500
117/117 [==============================] - 9s 79ms/step - loss:

In [40]:
model.save(filepath='model.h5')